# Decision Theory Project - Chess
*By Jelle Huibregtse and Aron Hemmes*

Below is a Chess environment build from scratch with an Agent based on reward

## 1. Setup
- Loading in some libraries
- Configuring layout and styling

In [1]:
# Libraries
import random
from enum import Enum
from typing import List
from IPython.display import display
from ipywidgets import Layout, Button, Box

# Layout
field_layout = Layout(width = '50px', height = '50px', margin = '0', padding = '0')
close_layout = Layout(width = '50px', margin = '0', padding = '0')
column_layout = Layout(flex_flow = 'column')

In [2]:
%%HTML
<style>
.button {
    outline: none !important;
    box-shadow: none !important;
    cursor: default;
}

.row .button, .promotion_window .button:nth-child(-n+4) {
    font-size: 350%;
}

.white {
    color: white !important;
}

.selectable {
    cursor: pointer;
}

.board {
    padding: 5px 0;
    margin: 0 !important;
}

.output_subarea {
    padding: 0 !important;
}

.promotion_window {
    border-radius: 0 0 3px 3px;
    box-shadow: 3px 3px 10px rgb(0 0 0 / 45%);
    width: 50px;
    position: absolute;
    z-index: 1000;
    margin-top: 5px;
    background-color: var(--jp-layout-color3);
}

.promotion_window .button {
    background-color: transparent;
}

.promotion_window .button.close {
    background-color: transparent;
    height: 25px;
    line-height: 25px;
}

.promotion_window.reverted {
    flex-direction: column-reverse !important;
    border-radius: 3px 3px 0 0;
    box-shadow: 3px -3px 10px rgb(0 0 0 / 45%);
}

.circle_filled::before {
    content: '';
    width: 40%;
    height: 40%;
    border-radius: 50%;
    position: absolute;
    background: rgb(51, 82, 107);
    top: 30%;
    left: 30%;
}

.circle::before {
    content: '';
    width: 100%;
    height: 100%;
    border: rgb(51, 82, 107) 5px solid;
    border-radius: 50%;
    position: absolute;
    top: 0;
    left: 0;
}

.selected::before {
    content: '';
    width: 100%;
    height: 100%;
    border: rgb(162, 185, 204) solid 5px;
    position: absolute;
    top: 0;
    left: 0;
}
</style>

## 2. Definition of the Environment

the code below defines all characteristics of a Chess Environment:

In [3]:
class Color(Enum):
    WHITE = 1
    BLACK = 2

class PieceType(Enum):
    KING = '♚'
    QUEEN = '♛'
    ROOK = '♜'
    BISHOP = '♝'
    KNIGHT = '♞'
    PAWN = '♟︎'

class Action(Enum):
    Move = 1
    Take = 2
    Castle = 3
    EnPassant = 4

class Move:
    def __init__(self, action : Action, x_start : int, y_start : int, x_end : int, y_end : int, turn: int, promoting : PieceType = None):
        self.action = action
        self.x_start = x_start
        self.y_start = y_start
        self.x = x_end
        self.y = y_end
        self.turn = turn
        self.promoting = promoting

class Piece:
    def __init__(self, x : int, y : int, type : PieceType, color : Color):
        self.x = x
        self.y = y
        self.color = color
        self.type = type
        self.history = []
    
    def execute_move(self, move : Move):
        self.history.append(move)
        self.x = move.x
        self.y = move.y
        self.type = move.promoting if not move.promoting == None else self.type
    
    def set_pos(self, x, y):
        self.x = x
        self.y = y
    
    def take(self):
        self.x = None
        self.y = None
    
    def get_legal_moves(self, board) -> List[Move]:
        return MoveGenerator().generate_legal_moves(board, self)

class ChessEnvironment:
    def __init__(self, engine : any = None, player_color : Color = Color.WHITE, pieces : List[Piece] = None):
        self.engine = engine
        self.player_color = player_color
        self.pieces = pieces if not pieces == None else self.generate_default()
        self.turn = 0
        self.rendered_board = None
        
        # Execute opposing engine
        if not self.engine == None and player_color == Color.BLACK:
            self.engine(self)
    
    def get_active_color(self) -> Color:
        return Color.WHITE if self.turn % 2 == 0 else Color.BLACK
    
    def reset(self):
        self.pieces = self.generate_default()
        self.player_color = Color.BLACK if self.player_color == Color.WHITE else Color.BLACK
        self.turn = 0
        
        # Update rendered board
        if not self.rendered_board == None:
            self.rendered_board.update(self)

    def clear(self):
        self.pieces = []
        self.turn = 0
        
        # Update rendered board
        if not self.rendered_board == None:
            self.rendered_board.update(self)

    def get_piece(self, x, y) -> Piece:
        pieces = [piece for piece in self.pieces if piece.x == x and piece.y == y]
        return pieces[0] if len(pieces) > 0 else None

    def execute_move(self, move : Move):
        piece = self.get_piece(move.x_start, move.y_start)
        
        # Get targeted piece
        target_piece = None
        if move.action == Action.Take:
            target_piece = self.get_piece(move.x, move.y)
        elif move.action == Action.EnPassant:
            target_piece = self.get_piece(move.x, piece.y)
        elif move.action == Action.Castle:
            target_piece = self.get_piece(0 if move.x < move.x_start else 7, move.y)
        
        # Move the rook when castling
        if move.action == Action.Castle:
            target_piece.set_pos(move.x + (1 if move.x < move.x_start else -1), target_piece.y)
        
        # Take piece if there's a target  
        elif not target_piece == None:
            target_piece.take()
        
        # If a pawn is on the final rank and move doesn't have a promoting piece set promoting piece to queen
        if piece.type == PieceType.PAWN and move.promoting == None and move.y == (7 if self.player_color == Color.WHITE else 0):
            move.promoting = PieceType.QUEEN
        
        # Execute the move
        piece.execute_move(move)
        
        # Update rendered board
        if not self.rendered_board == None:
            self.rendered_board.update(self)
        
        # Increment board turn by 1
        self.turn += 1
        
        # Execute opposing engine
        if not self.engine == None and self.turn % 2 == (1 if self.player_color == Color.WHITE else 0):
            self.engine(self)

    def generate_default(self) -> List[Piece]:
        # Generate the default 8 * 8 chess board pieces
        side = ['♜', '♞', '♝', '♛', '♚',  '♝', '♞', '♜'] + ['♟︎' for _ in range(8)]
        p = side + [' ' for _ in range(32)] + side[::-1]
        pieces = []
        for i in range(len(p)):
            piece = p[i]
            if piece != ' ':
                x = i % 8
                y = int(i / 8)
                pieces.append(Piece(x, y, PieceType(piece), Color.BLACK if i < 32 else Color.WHITE))
                
        return pieces
    
    def render(self):
        self.rendered_board = DisplayBoard(self)

## 3. Valid move generator

In [4]:
class MoveGenerator:
    # A dynamic list of legal moves for a piece
    def generate_legal_moves(self, board : ChessEnvironment, piece: Piece) -> List[Move]:
        # Generate all pseudo legal moves
        moves = self.generate_pseudo_legal_moves(board, piece)
        
        # Remove moves that put or leave king in check
        # TODO: LEGAL MOVES FOR KING
        # if piece.type == PieceType.KING:
        
        return moves
    
    # Pseudo-legal moves might leave or put the king in check and allows castling while king is in check, but are otherwise valid
    def generate_pseudo_legal_moves(self, board : ChessEnvironment, piece: Piece) -> List[Move]:
        moves = []
        
        if piece.type == PieceType.PAWN:
            for x, y in [[piece.x + x, piece.y + (y if not piece.color == Color.WHITE else -y)] for x, y in ([[0, 1], [1, 1], [-1, 1]] + ([[0, 2]] if len(piece.history) == 0 else []))]:
                if 7 >= x >= 0 and 7 >= y >= 0:
                    # If no pieces in between
                    if not abs(piece.y - y) == 2 or (abs(piece.y - y) == 2 and board.get_piece(x, int((piece.y + y) / 2)) == None):
                        target_piece = board.get_piece(x, y)
                        if x == piece.x and target_piece == None:
                            moves.append(Move(Action.Move, piece.x, piece.y, x, y, board.turn))
                        if not x == piece.x and not target_piece == None and not target_piece.color == piece.color:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
            
            # En Passant
            if (piece.y == 3 and piece.color == Color.WHITE) or (piece.y == 4 and piece.color == Color.BLACK):
                for x, y in [[piece.x + x, piece.y + (y if not piece.color == Color.WHITE else -y)] for x, y in [[1, 1], [-1, 1]]]:
                    target_piece = board.get_piece(x, piece.y)
                    if not target_piece == None and target_piece.type == PieceType.PAWN:
                        move = target_piece.history[-1] if len(target_piece.history) > 0 else None
                        if not move == None and move.turn == board.turn - 1 and abs(move.y_start - move.y) == 2:
                            moves.append(Move(Action.EnPassant, piece.x, piece.y, x, y, board.turn))
        
        elif piece.type == PieceType.KNIGHT:
            for m in [1, -1]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for x, y in [[1, 2], [-1, 2], [2, 1], [-2, 1]]]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None or (not target_piece == None and not target_piece.color == piece.color):
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
        
        elif piece.type == PieceType.KING:
            for m in [1, -1]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for x, y in [[0, 1], [1, 1], [1, 0], [1, -1]]]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None or (not target_piece == None and not target_piece.color == piece.color):
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
            
            # Castling
            if len(piece.history) == 0:
                for x, y in [[0, piece.y], [7, piece.y]]:
                    target_piece = board.get_piece(x, y)
                    
                    # If no pieces in between
                    if not any([p for p in [board.get_piece(x, y) for x in range(min(x, piece.x) + 1, max(x, piece.x))] if not p == None]):
                        if not target_piece == None and target_piece.type == PieceType.ROOK and len(target_piece.history) == 0:
                            a, b = [piece.x - 2 if x < piece.x else piece.x + 2, piece.y]
                            if 7 >= a >= 0 and 7 >= b >= 0:
                                moves.append(Move(Action.Castle, piece.x, piece.y, a, b, board.turn))
        
        elif piece.type == PieceType.ROOK or piece.type == PieceType.QUEEN:
            for x, y in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for m in range(1, 8)]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
                        elif not target_piece == None and not target_piece.color == piece.color:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
                            break
                        else:
                            break
        
        if piece.type == PieceType.BISHOP or piece.type == PieceType.QUEEN:
            for x, y in [[1, 1], [-1, -1], [1, -1], [-1, 1]]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for m in range(1, 8)]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
                        elif not target_piece == None and not target_piece.color == piece.color:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
                            break
                        else:
                            break
        
        return moves

## 4. Custom chess board

Takes care of rendering and interacting with a visual board.

In [5]:
class DisplayBoard:
    def __init__(self, board : ChessEnvironment):
        self.board = board
        self.fields = []
        self.player_selected_moves = []
        self.player_selected_move = None

        # Add promotion window
        promotion = self.generate_promotion_window()
        self.promotion_window = promotion

        # Add grid
        rows = []
        spaces = (list(range(8)) if self.board.player_color == Color.WHITE else list(range(8))[::-1])
        for y in spaces:
            row = Box([self.generate_field(x, y) for x in spaces])
            row.add_class('row')
            rows.append(row)

        grid = Box(children = rows, layout = column_layout)
        grid.add_class('board')

        # Display elements
        display(promotion)
        display(grid)

    def close_promotion_window(self, _):
        if hasattr(self, 'promotion_window'):
            self.promotion_window.add_class('hidden')

    def open_promotion_window(self, field: int):
        if hasattr(self, 'promotion_window'):
            # Color the buttons on promotion window
            if hasattr(self, 'promotion_window_buttons'):
                for button in self.promotion_window_buttons:
                    button.add_class('white') if self.board.get_active_color() == Color.WHITE else button.remove_class('white')
            
            # Place promotion window in correct location
            self.promotion_window.remove_class('reverted')
            margin = '5px 0 0 ' + str((field) * 50) + 'px'
            if field > 7:
                self.promotion_window.add_class('reverted')
                field = 7 - (63 - field)
                margin = str(5 + 3.5 * 50) + 'px 0 0 ' + str((field) * 50) + 'px'
            
            self.promotion_window.layout.margin = margin
            
            # Show promotion window
            self.promotion_window.remove_class('hidden')

    def promote_piece(self, e):
        
        if not self.player_selected_move == None:
            # Add promotion type to move
            move = self.player_selected_move
            move.promoting = PieceType(e.description)

            # Execute move
            self.board.execute_move(move)
        
        # Hide promotion window
        self.close_promotion_window('')

    def field_click(self, e):
        # Handles clicking on fields
        # Hide promotion window
        self.close_promotion_window('')
        
        # Hide selection and move indicators
        for field in self.fields:
            field.remove_class('circle_filled')
            field.remove_class('circle')
            field.remove_class('selected')
        
        # Check if it's the player's turn
        if self.board.get_active_color() == self.board.player_color or self.board.engine == None:
            # Clicked location variables
            clicked_field = int(e.tooltip)
            clicked_x = clicked_field % 8
            clicked_y = int(clicked_field / 8)

            # Getting move on clicked location
            move = [move for move in self.player_selected_moves if move.x == clicked_x and move.y == clicked_y]
            move = move[0] if any(move) else None

            # Open promotion window
            promotion = False
            piece = self.board.get_piece(move.x_start, move.y_start) if not move == None else None
            if not piece == None and piece.type == PieceType.PAWN and move.y == (0 if self.board.get_active_color() == Color.WHITE else 7):
                self.player_selected_move = move
                self.open_promotion_window(clicked_field if self.board.player_color == Color.WHITE else 63 - clicked_field)
                promotion = True

            # Execute move
            if not promotion and not move == None:
                self.board.execute_move(move)

            # Add moves for clicked piece
            if not promotion and move == None:
                clicked_piece = self.board.get_piece(clicked_x, clicked_y)
                if not clicked_piece == None and clicked_piece.color == self.board.get_active_color() and (len(self.player_selected_moves) == 0 or not (self.player_selected_moves[0].x_start == clicked_x and self.player_selected_moves[0].y_start == clicked_y)):
                    # Add clicked piece's moves to player_selected_moves
                    self.player_selected_moves = clicked_piece.get_legal_moves(self.board)

                    # Show selected outline
                    field = clicked_piece.x + clicked_piece.y * 8 if self.board.player_color == Color.WHITE else 63 - (clicked_piece.x + clicked_piece.y * 8)
                    self.fields[field].add_class('selected')

                    # Show move indicators
                    for move in self.player_selected_moves:
                        f = move.x + move.y * 8
                        if self.board.get_piece(move.x, move.y) == None:
                            self.fields[f if self.board.player_color == Color.WHITE else 63 - f].add_class('circle_filled')
                        else:
                            self.fields[f if self.board.player_color == Color.WHITE else 63 - f].add_class('circle')
                else:
                    self.player_selected_moves = []
            else:
                self.player_selected_moves = []

    def render_piece(self, field, piece = None):
        if piece == None:
            piece = self.board.get_piece(field % 8, int(field / 8))
        
        f = field if self.board.player_color == Color.WHITE else 63 - field
        self.fields[f].description = piece.type.value if not piece == None else ' '
        if not piece == None and piece.color == Color.WHITE:
            self.fields[f].add_class('white')
        else:
            self.fields[f].remove_class('white')
        
        if not piece == None and self.board.player_color == piece.color:
            self.fields[f].add_class('selectable')
        else:
            self.fields[f].remove_class('selectable')

    def generate_field(self, x, y) -> Button:
        field = x + y * 8
        btn = Button(layout = field_layout, tooltip = str(field))
        # Setting grid color
        btn.style = {'button_color': '#7495b1' if (x + y) % 2 == 0 else '#477397'}
        btn.add_class('button')
        btn.on_click(self.field_click)
        self.fields.append(btn)
        self.render_piece(field)
        
        return self.fields[-1]

    def generate_promotion_window(self):
        # Generate a promotion window for promoting pawns
        buttons = []
        for x in range(4):
            piece = '♞' if x == 1 else '♜' if x == 2 else '♝' if x == 3 else '♛'
            btn = Button(layout = field_layout, description = piece)
            btn.add_class('button')
            btn.add_class('selectable')
            btn.on_click(self.promote_piece)
            buttons.append(btn)
        close = Button(layout = close_layout, description = '🗙')
        close.add_class('button')
        close.add_class('selectable')
        close.add_class('close')
        close.on_click(self.close_promotion_window)
        self.promotion_window_buttons = buttons + [close]
        buttons.append(close)
        promotion = Box(children = buttons, layout = column_layout)
        promotion.add_class('promotion_window')
        promotion.add_class('hidden')
        
        return promotion

    # Rendering a custom chess board
    def update(self, board: ChessEnvironment):
        # re-render of the board
            for i in range(len(self.fields)):
                self.render_piece(i)

            # Add pointer cursor to selectable pieces
            if self.board.engine == None:
                for f in range(len(self.fields)):
                    p = self.board.get_piece(f % 8, int(f / 8))
                    if not p == None and p.color == self.board.get_active_color():
                        self.fields[f].add_class('selectable')
                    else:
                        self.fields[f].remove_class('selectable')

## 5. No Agent (player vs player)

In [6]:
board = ChessEnvironment()
board.render()

Box(children=(Button(description='♛', layout=Layout(height='50px', margin='0', padding='0', width='50px'), sty…

Box(children=(Box(children=(Button(description='♜', layout=Layout(height='50px', margin='0', padding='0', widt…

## 6. Random Agent

In [7]:
def random_engine(board : ChessEnvironment):
    move_generator = MoveGenerator()

    # Get all valid moves
    pieces = [moves for moves in [move_generator.generate_legal_moves(board, piece) for piece in board.pieces if not piece.color == board.player_color and not piece.x == None and not piece.y == None] if len(moves) > 0]
    
    # If there's a move  available
    if len(pieces) > 0:
        # Choose a random piece then coose a random move for this piece
        moves = random.choice(pieces)
        move = random.choice(moves)

        # Execute the move
        board.execute_move(move)

# Create board with engine and render
board = ChessEnvironment(random_engine)
board.render()

Box(children=(Button(description='♛', layout=Layout(height='50px', margin='0', padding='0', width='50px'), sty…

Box(children=(Box(children=(Button(description='♜', layout=Layout(height='50px', margin='0', padding='0', widt…